In [1]:
import preprocess as pp
import configs
import persister
import relation
import lda

import re
import json

from stanfordcorenlp import StanfordCoreNLP
from nltk import sent_tokenize, word_tokenize, pos_tag
from nltk.corpus import wordnet
from nltk.tree import Tree
from imp import reload

not use stanford CoreNLP client!


In [ ]:
# 文档向量化测试
# count vectorizer

In [ ]:
a = "Many existing knowledge bases(KBs), including Freebase, Yago, and NELL, rely on a ﬁxed ontology, given as an input to the system, which deﬁnes the data to be cataloged in the KB, i.e., a hierarchy of categories and relations between them. The system then extracts facts that match the predeﬁned ontology. We propose an unsupervised model that jointly learns a latent ontological structure of an input corpus, and identiﬁes facts from the corpus that match the learned structure. Our approach combines mixed membership stochastic block models and topic models to infer a structure by jointly modeling text, a latent concept hierarchy, and latent semantic relationships among the entities mentioned in the text. As a case study, we apply the model to a corpus of Web documents from the software domain,and evaluate the accuracy of the various components of the learned ontology. "
b = ["hello world. what the fuck."]

In [ ]:
c = [" ".join(preprocess(a))]
vector = CountVectorizer(ngram_range=(1, 2), vocabulary=["knowledge base"], stop_words='english')
vector.build_analyzer()
x = vector.fit_transform(c)

In [ ]:
estimator = CountVectorizer(ngram_range=(1, 2), vocabulary=['fuck', 'hello', 'world', "hello world"], stop_words='english')
estimator.build_analyzer()
res = estimator.fit_transform(b)

In [ ]:
estimator.get_feature_names()

In [ ]:
res.toarray()
# estimator.vocabulary_

In [ ]:
# 斯坦福nlp工具测试
# stanfordcorenlp

In [ ]:
nlp = StanfordCoreNLP('/mnt/d/stanford-corenlp-full-2018-02-27', port=9000)
sentence = 'Brink is taking part in the final exercise.'
print('Tokenize:', nlp.word_tokenize(sentence))
print('Part of Speech:', nlp.pos_tag(sentence))
print('Named Entities:', nlp.ner(sentence))
print('Constituency Parsing:', nlp.parse(sentence))
print('Dependency Parsing:', nlp.dependency_parse(sentence))

nlp.close() # Do not forget to close! The backend server will consume a lot memery.

In [ ]:
# 官方corenlp api
# stanza

In [ ]:
# 新闻数据集测试
# 20newsgroups

In [2]:
import newsgroups
size = 500
tmp = newsgroups.get_news_data(size)
newsdata = []
for cate in tmp:
    newsdata.extend(cate)

In [21]:
from stanza.server import CoreNLPClient
with CoreNLPClient(properties="./corenlp_server.props", timeout=30000, memory='5G') as client:
    res = relation.corenlp_annotate(client, pp.format_news(newsdata[0]))

Setting server defaults from: ./corenlp_server.props
Starting server with command: java -Xmx5G -cp /mnt/d/stanford-corenlp-full-2018-02-27/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 30000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties ./corenlp_server.props -preload pos,parse,depparse,lemma


In [ ]:
texts = [' '.join(pp.preprocess_abstract(a))
persister.save_json(configs.NEWSDATA, texts)

In [ ]:
# 处理成句子
# nltk.sent_tokenize

In [ ]:
newssent = []
for news in newsdata:
    newssent.append(pp.split2sent(news))
persister.save_json(configs.NEWSSENT, newssent)

In [ ]:
newssenttoken = []
for news in newssent:
    tokenized_lemmatized_news = []
    for sent in news:
        tokenized_lemmatized_news.append(relation.lemmatize_sent_words(sent))
    newssenttoken.append(tokenized_lemmatized_news)
persister.save_json(configs.NEWSSENTTOKEN, newssenttoken)

In [ ]:
# load preprossed data

In [11]:
rawnews = persister.load_json(configs.RAWNEWS)
newsparse = persister.read_parse()

In [12]:
# load lda res
lda_input = persister.read_input(configs.NEWSINPUT)
terms, doc_topic, topic_word = persister.read_lda(configs.NEWSLDA)
lda.print_topics(topic_word, terms, doc_topic)
top_terms, top_docs = lda.get_topics(topic_word, terms, doc_topic)

# 1 --------------------
rrb:2983.9153838742063
lrb:2720.8698595867554
write:899.1036169748056
car:764.8572207028628
article:677.2676219610042
year:652.406779999483
think:518.8965484921148
like:470.789389436032
game:442.18502129581856
good:432.498193517502
say:414.8350138122594
run:402.83013337173657
look:383.26345258441614
know:370.85754341543856
make:355.22744451649396
time:344.86500759451076
team:338.16545263803675
win:310.2351674294442
use:285.9389771948374
better:265.82719878526507
1857:0.9992840173174373
1624:0.9988674712694471
1538:0.9982671028536346
1750:0.9982029162768897
1925:0.998089983660114
1628:0.9979540071768394
1743:0.9978358508755351
1456:0.9977147942096349
1656:0.9976056334423457
1195:0.9975755884518485
1521:0.997554954313495
1577:0.997522031138196
1923:0.9974954232505238
1062:0.9972832299018409
1927:0.9972713598479651
1018:0.9972554252325445
1586:0.9972115930009625
1893:0.9971313763132627
1512:0.9970376279803703
1568:0.9968700654429791
# 2 --------------------
rrb:45

In [12]:
idx_news = 1195
idx_sent = 40

In [ ]:
print(pos_tag(word_tokenize(newssent[idx_news][43])))
print(pp.WNL.lemmatize("has", wordnet.VERB))

In [ ]:
print(len(newsparse[idx_news]["sentences"]))
print(newsparse[idx_news]["sentences"][idx_sent].keys())
examparse = newsparse[idx_news]["sentences"][idx_sent]
print(examparse["enhancedPlusPlusDependencies"])

In [ ]:
tt = Tree.fromstring(examparse["parse"])
tt.pretty_print()

In [ ]:
t4 = [t[0] for t in top_terms[3]]
t4_values = [t[1] for t in top_terms[3]]
print(t4)
print(t4_values)
idxs, importance, count = relation.extract_important_sents(newssenttoken[idx_news], t4, t4_values)
print(idxs, importance[40], count[40])

In [ ]:
relation.extract_word_relation_from_sent(i, newsparse[1200]["sentences"][sent_idx]["enhancedDependencies"])

In [ ]:
for term in top_terms[3]:
    for news in top_docs[3]:
        time.sleep(2)
        news_idx = news[0]
        for sent_idx,sent in enumerate(newssenttoken[news_idx]):
            if type(newsparse[news_idx]) is str:
                print(news_idx, "parse", "err")
                break
            topic_word_idxs = relation.get_word_idx(term[0], sent)
            for i in topic_word_idxs:
                print(sent_idx, i, news_idx)
                rs = relation.extract_word_relation_from_sent(i, newsparse[news_idx]["sentences"][sent_idx]["enhancedDependencies"])
                for r in rs:
                    print(relation.convert_relation2str(r))

In [ ]:
t = Tree.fromstring(newsparse[idx_news]["sentences"][5]["parse"])
t.pretty_print()

In [4]:
# abstract data

In [2]:
raw_data = persister.load_json(configs.RAWABSTRACT)
abslda_input = persister.read_input(configs.ABSTRACTINPUT)
terms, doc_topic, topic_word = persister.read_lda(configs.ABSTRACTLDA)
absparse = persister.read_parse(configs.ABSTRACTPARSE)

In [10]:
lda.print_topics(topic_word, terms, doc_topic)
top_terms, top_docs = lda.get_topics(topic_word, terms, doc_topic)

# 1 --------------------
topic:156.1207266700688
model:123.6125417658068
latent:81.81636740472715
drive:68.39228141652279
lda:66.31282489616528
algorithm:57.973691649739415
tv:57.169341660507094
result:54.61468279221959
structure:52.354525594703105
datum:51.27169147312781
cluster:50.37103136202075
group:48.95385475767535
clustering:47.963388441469576
propose:46.146326739241516
allocation:44.40419936971626
protein:43.81142090868558
problem:42.86810620194955
probability:39.61799552703611
state:37.93653534497001
analysis:37.88997983208092
1097:0.9909523807882038
1422:0.9886904759336536
420:0.9452298386941872
81:0.9277537269354105
2390:0.7950361209139868
2065:0.7292551747222003
2136:0.7292551747222003
199:0.7262216087074623
1383:0.6790919464713469
2400:0.6772698071243939
2474:0.6451203754635143
854:0.6253382226913716
1633:0.6100936738250698
1928:0.5989132865554739
1642:0.5914175539965929
2455:0.5859637905774294
2541:0.5853316688906237
2184:0.5682706602066767
1439:0.5497041737925908
977:0.5